In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re

In [2]:
def load_data():
    data = pd.read_csv('Covid_Delta_Varian.csv')#ubah nama file sesai dengan nama file kalian
    return data

In [3]:
tweet_df = load_data()
tweet_df.head()

,waktu,user,text
0,6/23/2021 5:12,🦅,b'RT @BL4rrrrr: Lonjakan kasus pasca Lebaran t...
1,6/23/2021 5:12,𝓣𝓱𝓮 𝐆𝐔𝐀𝐑𝐃𝐈𝐀𝐍,b'RT @BL4rrrrr: Lonjakan kasus pasca Lebaran t...
2,6/23/2021 5:12,akui saja,b'RT @afrkml: Be alert ya krn semua gejala yg ...
3,6/23/2021 5:11,cuba 🦄,b'Covid varian delta ini lebih bandel dari yg ...
4,6/23/2021 5:07,ZulkarnainJailie,b'RT @bharianmy: #BHdunia AS diramal gagal men...


In [5]:
df = pd.DataFrame(tweet_df[['waktu', 'user', 'text']])

In [6]:
#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [20]:
df.head(5)

,waktu,user,text,remove_user,remove_http
10588,6/16/2021 10:41,Budi Santoso.,"b""@OposisiCerdas Mungkinkah krn UGD Puskesmas ...","b"" Mungkinkah krn UGD Puskesmas belum siap tan...","b"" Mungkinkah krn UGD Puskesmas belum siap tan..."
11564,6/15/2021 4:10,Kezia Karamoy,"b""@null Varian Delta Covid-19 Bermutasi Jadi '...","b"" Varian Delta Covid-19 Bermutasi Jadi 'Delta...","b"" Varian Delta Covid- Bermutasi Jadi 'Delta P..."
11557,6/15/2021 4:19,netizen,"b""@hallidii said: Kalau bermutasi lagi jadi --...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ..."
2479,6/22/2021 3:31,MSNIndonesia,"b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid- V..."
9014,6/19/2021 7:15,CNBC Indonesia,"b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C..."


In [9]:
#CLEANSING

def remove(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))
df.sort_values("remove_http", inplace = True)
df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)

In [15]:
df.tail(5)

,waktu,user,text,remove_user,remove_http
1613,6/22/2021 7:15,◢ ◤,b'varian delta covid 19 udah ada dibatam aja :(',b'varian delta covid 19 udah ada dibatam aja :(',b'varian delta covid udah ada dibatam aja :('
4862,6/21/2021 15:25,daffa🔰,"b'varian delta iki bangsat ancene, hampir seti...","b'varian delta iki bangsat ancene, hampir seti...","b'varian delta iki bangsat ancene, hampir seti..."
9645,6/18/2021 7:32,wisnu nugroho,b'waspada ya gaes &gt;&gt;&gt; Peneliti Ungkap...,b'waspada ya gaes &gt;&gt;&gt; Peneliti Ungkap...,b'waspada ya gaes &gt;&gt;&gt; Peneliti Ungkap...
9932,6/17/2021 17:28,mas al,b'wkwkwkw indonesia resmi gelombang 2 covid ni...,b'wkwkwkw indonesia resmi gelombang 2 covid ni...,b'wkwkwkw indonesia resmi gelombang covid nih...
9202,6/19/2021 2:22,HanG_Nadin,b'~Jokowi Lawan Pandemi\nWamenkes Dante Sakson...,b'~Jokowi Lawan Pandemi\nWamenkes Dante Sakson...,b'~Jokowi Lawan Pandemi\nWamenkes Dante Sakson...


In [16]:
#STOPWORD

from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [21]:
df.head(5)

,waktu,user,text,remove_user,remove_http
10588,6/16/2021 10:41,Budi Santoso.,"b""@OposisiCerdas Mungkinkah krn UGD Puskesmas ...","b"" Mungkinkah krn UGD Puskesmas belum siap tan...","b"" Mungkinkah krn UGD Puskesmas belum siap tan..."
11564,6/15/2021 4:10,Kezia Karamoy,"b""@null Varian Delta Covid-19 Bermutasi Jadi '...","b"" Varian Delta Covid-19 Bermutasi Jadi 'Delta...","b"" Varian Delta Covid- Bermutasi Jadi 'Delta P..."
11557,6/15/2021 4:19,netizen,"b""@hallidii said: Kalau bermutasi lagi jadi --...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ..."
2479,6/22/2021 3:31,MSNIndonesia,"b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid- V..."
9014,6/19/2021 7:15,CNBC Indonesia,"b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C..."


In [18]:
#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [22]:
df.head(5)

,waktu,user,text,remove_user,remove_http
10588,6/16/2021 10:41,Budi Santoso.,"b""@OposisiCerdas Mungkinkah krn UGD Puskesmas ...","b"" Mungkinkah krn UGD Puskesmas belum siap tan...","b"" Mungkinkah krn UGD Puskesmas belum siap tan..."
11564,6/15/2021 4:10,Kezia Karamoy,"b""@null Varian Delta Covid-19 Bermutasi Jadi '...","b"" Varian Delta Covid-19 Bermutasi Jadi 'Delta...","b"" Varian Delta Covid- Bermutasi Jadi 'Delta P..."
11557,6/15/2021 4:19,netizen,"b""@hallidii said: Kalau bermutasi lagi jadi --...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ..."
2479,6/22/2021 3:31,MSNIndonesia,"b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid- V..."
9014,6/19/2021 7:15,CNBC Indonesia,"b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C..."


In [23]:
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [24]:
df.head(5)

,waktu,user,text,remove_user,remove_http,tweet_clean
10588,6/16/2021 10:41,Budi Santoso.,"b""@OposisiCerdas Mungkinkah krn UGD Puskesmas ...","b"" Mungkinkah krn UGD Puskesmas belum siap tan...","b"" Mungkinkah krn UGD Puskesmas belum siap tan...","[b, krn, ugd, puskesmas, tangan, varian, covid..."
11564,6/15/2021 4:10,Kezia Karamoy,"b""@null Varian Delta Covid-19 Bermutasi Jadi '...","b"" Varian Delta Covid-19 Bermutasi Jadi 'Delta...","b"" Varian Delta Covid- Bermutasi Jadi 'Delta P...","[b, varian, delta, covid, mutasi, delta, plus,..."
11557,6/15/2021 4:19,netizen,"b""@hallidii said: Kalau bermutasi lagi jadi --...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ...","b"" said: Kalau bermutasi lagi jadi --- 'Delta ...","[b, said, mutasi, delta, plus, plus, n, nimdon..."
2479,6/22/2021 3:31,MSNIndonesia,"b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid-19...","b""Dikenal Lebih 'Ganas', Inilah Virus Covid- V...","[b, kenal, ganas, virus, covid, varian, delta,..."
9014,6/19/2021 7:15,CNBC Indonesia,"b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C...","b""Fakta Seputar Varian Baru Corona Delta Si 'C...","[b, fakta, putar, varian, corona, delta, si, c..."


In [ ]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [ ]:
df.sort_values("Tweet", inplace = True)
df.drop(df.columns[[0,1,2,3,4]], axis = 1, inplace = True)
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)
df.to_csv('output.csv',encoding='utf8', index=False)
df